In [1]:
class GomokuAgent:
    def __init__(self, ID, BOARD_SIZE, X_IN_A_LINE):
        self.ID = ID
        self.BOARD_SIZE = BOARD_SIZE
        self.X_IN_A_LINE = X_IN_A_LINE

    def move(self, board):
        return (0,0)

In [2]:
import numpy as np
from misc import legalMove
from misc import winningTest
from gomokuAgent import GomokuAgent
from queue import PriorityQueue
from time import time as time

class Player(GomokuAgent):
    def __init__(self, ID, BOARD_SIZE, X_IN_A_LINE):
        self.ID = ID
        self.BOARD_SIZE = BOARD_SIZE
        self.X_IN_A_LINE = X_IN_A_LINE
        self.board = np.zeros((self.BOARD_SIZE, self.BOARD_SIZE), dtype=int)
        self.root = None
        self.current_node = None
        self.turn = 0
        self.is_set = False
        self.centre = int(BOARD_SIZE/2)
        self.op_moves = [] 
        self.my_moves = []
        
        
    def move(self,board):
        self.turn += 1
        move_loc = (1,1)
        
        if not self.is_set:                                         #if it is our first turn we must initialise
           
            first_move = self.first_move(board[:])                  #our first move can just be a precomputed central move
            
            self.current_node = self.root
        else:
            op_move = self.get_op_move(board)
            self.op_moves.append(op_move)                           #else we should get the op's move so we can move down the tree
            self.current_node = self.get_node(op_move)
            
        
        self.board = board.copy()
        self.explore_node(self.current_node,4)
        print("start board")
        print(self.current_node.board)
        print("e.g deepest board")
        print(self.current_node.children[0].children[0].children[0].children[0].board)
        
        
        if not self.is_set:
            self.is_set = True
            self.my_moves.append(first_move)
            self.board[first_move] = self.ID
            return first_move
        else:
            self.my_moves.append(move_loc)
            self.current_node = self.get_node(move_loc)
            self.board[move_loc] = self.ID
            return move_loc
            
    def get_root(self):
        return self.root
    
    def first_move(self,_board):                                    #initialises the root and makes the first move 
        my_first_move = (self.centre,self.centre)
        
        if not np.count_nonzero(_board):                            #the root is our move if the board is empty
            _board[my_first_move] = self.ID
            self.root = node(my_first_move, ID = self.ID,           
                             board = _board)
            return my_first_move                                    #we then return a central move as our first move
        
        else:                                                       #else the root is the opp's move
            
            op_first_move = self.get_op_move(_board)
            
            self.op_moves.append(op_first_move)
            
            self.root = node(op_first_move, ID = -self.ID,
                             board = _board)
            
            if my_first_move != op_first_move:
                return my_first_move                                #return our first move, or next to it if that is taken
            else:
                return (self.centre-1,self.centre)
                        
    def get_op_move(self,board):
        diff = board-self.board
        diff_arr = np.stack(np.nonzero(diff),axis= -1)
        
        op_move = (diff_arr[0][0],diff_arr[0][1])
        print("op made move: "+str(op_move))
        return op_move
        
    def get_node(self, move_loc):                                   #check if the node is in the tree (it is one of our predicted moves)
        for child in self.current_node.children:
            if child.move_pos == move_loc:
                return child
        child = node(move_loc,self.current_node,test=True)                    #else make a new node under the last node
        self.current_node.children.append(child)
        return child
            
    
    
    def explore_node(self,node,depth):
        if depth <= 1:
            return
        for child in node.children:
            if not child.children:
                child.order_children()
                self.explore_node(child,depth-1)
            else:
                self.explore_node(child,depth-1)
    
    
    
    
    

In [3]:
class node: #represents a board position in the tree
    count = 0
    
    def __init__(self, move_pos, parent = None, ID = None, board = None, test = False):
        node.count += 1
        
        self.move_pos = move_pos
        self.parent = parent
        self.children = []
        self.child_queue = PriorityQueue()
        self.board = []
        if parent == None:                                       #if no parent(root) initialise tree (run once per game)
            
            self.player_id = ID
            self.depth = 0
            self.plus_pieces = []
            self.minus_pieces = []
            self.empty_pieces = []
            for row in range(len(board)):
                for col in range(len(board)):
                    if board[row][col] == 0:
                        self.empty_pieces.append((row,col))
                    elif board[row][col] == 1:
                        self.plus_pieces.append((row,col))
                    elif board[row][col] == -1:
                        self.minus_pieces.append((row,col))
            self.score = 0
            self.board = board
            self.order_children()
                
        else:                                                     #we can gather info from our parent to save time re-computing it
            self.player_id = 0-self.parent.player_id
            self.depth = self.parent.depth+1
            
            self.plus_pieces = self.parent.plus_pieces.copy()
            self.minus_pieces = self.parent.minus_pieces.copy()
            self.empty_pieces = self.parent.empty_pieces.copy()
            
            self.board = self.parent.board.copy()
            try:
                self.empty_pieces.remove(self.move_pos)
            except: 
                print("tried to make node in taken position: "+str(self.move_pos))
            
            self.board[self.move_pos] = self.player_id              #the nodes' board is one piece different to the parent'
            
            if self.player_id == 1:
                self.plus_pieces.append(self.move_pos)
            else:
                self.minus_pieces.append(self.move_pos)
            self.score = 0
            
        if node.count % 10000 == 0:
            print(node.count)
            print(self.depth) 
            
            
    def order_children(self):
        if self.children == []:
            for blank in self.empty_pieces:
                
                heu = self.heuristic(blank)
                if heu > 0:
                    x_middle_pref = abs(blank[0]-5)
                    y_middle_pref = abs(blank[1]-5)
                    child = node(blank, self)
                    self.children.append(child)
                    self.child_queue.put((heu,x_middle_pref,y_middle_pref,node.count,child))
                    
    def heuristic(self, coords):
        #return 1
        score = 0
        for x in range(-1,2):
            for y in range(-1,2):
                try:
                    if self.board[coords[0]+x][coords[1]+y]!=0:
                        score+=1
                except:
                    pass
        return score
    
    def get_score(self):
        return 0
    
    def get_next(self,pos,direction):                              
        pass
        
    def evaluate(self):
        pass
    
    
            


In [4]:
#test case
"""
ID = 1
BOARD_SIZE = 11
X_IN_A_LINE = 5

p = Player(ID,BOARD_SIZE,X_IN_A_LINE)
empty_board = np.zeros((p.BOARD_SIZE, p.BOARD_SIZE), dtype=int)
board = empty_board[:]

test1 = time()
print(p.move(board)) #my move
board[(5,5)] = 1
print(str(time()-test1))

print("-------")
board[(6,5)] = -1 #op_move

test2 = time()
print(p.move(board)) #my move
print(str(time()-test2))
"""




'\nID = 1\nBOARD_SIZE = 11\nX_IN_A_LINE = 5\n\np = Player(ID,BOARD_SIZE,X_IN_A_LINE)\nempty_board = np.zeros((p.BOARD_SIZE, p.BOARD_SIZE), dtype=int)\nboard = empty_board[:]\n\ntest1 = time()\nprint(p.move(board)) #my move\nboard[(5,5)] = 1\nprint(str(time()-test1))\n\nprint("-------")\nboard[(6,5)] = -1 #op_move\n\ntest2 = time()\nprint(p.move(board)) #my move\nprint(str(time()-test2))\n'

In [5]:
test_board = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  #example board taken from https://en.wikipedia.org/wiki/Gomoku#/media/File:Gomoku-game-1.svg
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0,-1,-1,-1, 0, 0, 0, 0],
                       [ 0, 0, 0, 1, 1,-1, 1, 0, 0, 0, 0],
                       [ 0, 0, 0, 1,-1,-1,-1, 1, 0, 0, 0],
                       [ 0, 0, 0, 1,-1, 1, 0, 1, 0, 0, 0],
                       [ 0, 0, 1,-1,-1,-1, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [11]:
def get_next(board,pos,direction):                       #returns the id and position of the next cell in the line in tuple
        try:
            next_place = (pos[0]+direction[0],pos[1]+direction[1])
            return (board[next_place],next_place)
        except:
            return (2,next_place)
        
def evaluate(move_pos,board,ID):
    end_list = []
    line_list = []
    for x in range(-1,2):                                  #for each immediate neighbour
        for y in range(-1,2):
            if x!= 0 or y!=0:
                length = -1
                open_ends = 0
                centre = (board[move_pos],move_pos)               #get the neighbour
                end1 = get_next(board,centre[1],(x,y))
                end2 = get_next(board,centre[1],(x,y))
                cur_id = end2[0]
                if end1[0] != 0 and end1[0] != 2:
                    
                    while end1[0] == cur_id:
                        end1 = get_next(board,end1[1],(x,y))
                        length+=1
                        
                    if cur_id == ID:
                        while end2[0] == centre[0]:
                            end2 = get_next(board,end2[1],(-x,-y))
                            length+=1
                            
                    else:
                        length+=1
                        end2 = centre
                    
                    if end1[0] == 0:
                        open_ends+=1
                    
                    if end2[0] == 0:
                        open_ends+=1
                        
                    if length>1 and ((end1 not in end_list and end2 not in end_list) or end2 == centre):
                        line_list.append((length,open_ends,end1,end2,cur_id))
                        end_list.append(end1) 
                        end_list.append(end2) 
                
    return line_list

In [13]:
test_board = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  #example board taken from https://en.wikipedia.org/wiki/Gomoku#/media/File:Gomoku-game-1.svg
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0,-1,-1,-1, 0, 0, 0, 0],
                       [ 0, 0, 0, 1, 1,-1, 1, 0, 0, 0, 0],
                       [ 0, 0, 0, 1,-1,-1,-1, 1, 0, 0, 0],
                       [ 0, 0,-1, 1,-1, 1, 0, 1, 0, 0, 0],
                       [ 0, 0, 1,-1,-1,-1, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

###for TESTING

x = 4
y = 5
id_at_xy = -1
print("evaluating piece at (" + str(x)+","+str(y)+")'s effect on the score, is was placed by -1")
test = time()
lines = evaluate((x,y),test_board,id_at_xy)
for line in lines:
    print("line made: length ="+str(line[0])+" open ends ="+str(line[1])+" starting at: "
            +str(line[2][1])+ " ending at: "+str(line[3][1])+" for playerid: "+str(line[4]))
print("it took: " + str(time()-test))

evaluating piece at (4,5)'s effect on the score, is was placed by -1
line made: length =3 open ends =2 starting at: (4, 3) ending at: (4, 7) for playerid: -1
line made: length =2 open ends =0 starting at: (7, 2) ending at: (4, 5) for playerid: 1
line made: length =3 open ends =1 starting at: (7, 5) ending at: (3, 5) for playerid: -1
line made: length =2 open ends =1 starting at: (7, 8) ending at: (4, 5) for playerid: 1
it took: 0.001997709274291992
